In [1]:
%load_ext autoreload
%autoreload 2

from data_ingestion.read_data import train_df, test_df
from feature_engineering.feature_scaling import FeatureScaling
from feature_engineering.kmer_encoding import Kmer
from models.VotingClassifier import CustomVotingClassifier
from models.PLS import PLSDA

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, matthews_corrcoef
from models.LSTM import AttLSTM

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from gensim.models import KeyedVectors

import shap
import pandas as pd
import numpy as np


In [50]:
rare_aas = 'UOZB' 
train_df['epitope_name'] = train_df['epitope_name'].str.upper().replace(
    "UOZB", "X", regex=True)

test_df['epitope_name'] = test_df['epitope_name'].str.upper().replace(
    'UOZB', "X", regex=True)

In [116]:
scaler = FeatureScaling(['AAC', 'APAAC', 'PAAC', 'TPC'], 'PCA', 500)

raw_train = scaler.feature_encoder(train_df)
X_train = pd.DataFrame(scaler.feature_reduction(raw_train, train_df['label']))

raw_test = scaler.feature_encoder(test_df)
X_test = pd.DataFrame(scaler.feature_reduction(raw_test, test_df['label'], False))

y_train, y_test = train_df['label'], test_df['label']

In [117]:
model = CustomVotingClassifier(
                estimators=[
                    ('pls', PLSDA(n_components=15)),
                    ('log', LogisticRegression(C=0.0027059021490395217,
                                               max_iter=5000,
                                               penalty='l2',
                                               solver='liblinear',
                                               random_state=42)),
                    ('svm', SVC(C=0.001098881873199668,
                                kernel='rbf',
                                gamma=0.010951255895390475754,
                                probability=True,
                                random_state=42)),
                    ('rf', RandomForestClassifier(
                        n_estimators=401,
                        max_depth=40,
                        min_samples_split=7,
                        min_samples_leaf=1,
                        max_features='log2',
                        random_state=42
                    )),
                    ('xgb', XGBClassifier(
                        n_esimators=700,
                        max_depth=15,
                        random_state=42)),
                    ('knn', KNeighborsClassifier()),
                    
                    
                ],
                voting='soft',
                weights=[1, 1, 1, 1, 1, 1]
)

In [118]:
model.fit(X_train, y_train)

,estimators,"[('pls', ...), ('log', ...), ...]"
,voting,'soft'
,weights,"[1, 1, ...]"


In [119]:
test_results = []

y_test_pred = model.predict(X_test)

y_test_prob = model.predict_proba(X_test)[:, 1]

acc_test = model.score(X_test, y_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
specificity_test = tn / (tn + fp)
sensitivity_test = tp / (tp + fn)

auc_test = roc_auc_score(y_test, y_test_prob)
mcc_test = matthews_corrcoef(y_test, y_test_pred)

test_results.append({
    'ACCURACY': round(acc_test, 4),
    'SENSITIVITY': round(sensitivity_test, 4),
    'SPECIFICITY': round(specificity_test, 4),
    'AUC SCORE': round(auc_test, 4),
    'MCC SCORE': round(mcc_test, 4)
})

In [120]:
test_results

[{'ACCURACY': 0.7423,
  'SENSITIVITY': 0.7865,
  'SPECIFICITY': 0.7048,
  'AUC SCORE': 0.7832,
  'MCC SCORE': 0.49}]

In [108]:
X_train, X_test = Kmer(train_df, 3).encode_features(), Kmer(test_df, 3).encode_features()
y_train, y_test = train_df['label'], test_df['label']

In [109]:
model = KeyedVectors.load("models/protVec_100d_3grams.model", mmap='r')

def embed_protein_kmers(kmers_list, model, vector_size):
    zero_vec = np.zeros(vector_size, dtype=np.float32)
    embeddings = []

    for kmer in kmers_list:
        try:
            vec = model[kmer]  
        except KeyError:
            vec = zero_vec  
        embeddings.append(vec)

    embeddings = np.array(embeddings, dtype=np.float32)
    if embeddings.shape[0] == 0:
        return zero_vec  

    return np.array(embeddings).mean(axis=0)

X_train = np.array([embed_protein_kmers(seq, model, 100) for seq in X_train])
X_test = np.array([embed_protein_kmers(seq, model, 100) for seq in X_test])

In [110]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [111]:
scaler = FeatureScaling(['AAC', 'APAAC', 'PAAC', 'TPC'], 'PCA', 500)

raw_train = scaler.feature_encoder(train_df)
X_train = np.concatenate(
    [X_train, np.array(scaler.feature_reduction(raw_train, train_df['label']))],
    axis=1
)

raw_test = scaler.feature_encoder(test_df)
X_test = np.concatenate(
    [X_test, np.array(scaler.feature_reduction(raw_test, test_df['label'], False))],
    axis=1
)

y_train, y_test = train_df['label'], test_df['label']

In [97]:
X_train = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))

In [98]:
input_shape = (777, 500)

model = AttLSTM(input_shape=input_shape, learning_rate=1e-5)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test), 
    epochs=200,
    batch_size=32,
    verbose=1
)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5418 - loss: 0.6914 - val_accuracy: 0.5412 - val_loss: 0.6903
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6907 - val_accuracy: 0.5412 - val_loss: 0.6902
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5405 - loss: 0.6902 - val_accuracy: 0.5309 - val_loss: 0.6901
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6896 - val_accuracy: 0.5309 - val_loss: 0.6900
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6890 - val_accuracy: 0.5309 - val_loss: 0.6899
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6885 - val_accuracy: 0.5361 - val_loss: 0.6897
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6879 - val_accuracy: 0.5361 - val_loss: 0.6896
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5418 - loss: 0.6873 - val_accuracy: 0.5361 - 

In [ ]:
y_pred = model.predict(X_test)
y_pred
y_pred_classes = (y_pred > 0.5).astype("int32")

# Compute accuracy
acc = accuracy_score(y_test, y_pred_classes)
print(f"LSTM Model Accuracy: {acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
LSTM Model Accuracy: 0.6701
